In [68]:
from mnist import MNIST
mndata = MNIST('C:\\Users\\Divya\\Google Drive\\Acads\\Courses-Winter2018\\CSE253-NeuralNetworks\\neuralnets253\\HW1')
mndata.gz = True
images, labels = mndata.load_training() #Images is a list of 60k images of 784 dimensions, Labels is a list of 60k ints
testImages, testLabels = mndata.load_testing() #testImages is a list of 10k images of 784 dimensions, testLabels is a list of 10k ints

In [69]:
def initialize():
    #trainImages = images[:18000]
    #trainLabels = labels[:18000]
    trainImages = images[:100]
    trainLabels = labels[:100]
    validImages = images[18000:20000]
    validLabels = labels[18000:20000]
    testImgs = testImages[-2000:]
    testLbls = testLabels[-2000:]
    for img in trainImages:
        img.insert(0, 1)
    for img in testImgs:
        img.insert(0, 1)

In [70]:
initialize()
labelNums = len(set(trainLabels)) #total number of labels
dimensions = len(trainImages[0])
print(dimensions)

785


In [71]:
import numpy as np
weightVector = np.zeros((labelNums, dimensions))
l1Lambda = 0.1
l2Lambda = 0.1
learningRate = 0.001
def updateLambdas(l1Lam, l2Lam):
    global l1Lambda, l2Lambda
    l1Lambda = l1Lam
    l2Lambda = l2Lam

In [72]:
def updateWeighti(i, l1):
    global weightVector, l1Lambda, l2Lambda, learningRate
    predDiff = np.transpose(trainLabels - np.matmul(trainImages, np.transpose(weightVector[i])));
    if(not l1):
        weightVector[i] = weightVector[i] + learningRate*(np.matmul(predDiff, trainImages) - 2*l2Lambda*weightVector[i]);
    else:
        l1RegAdd = [1 if x>0 else -1 for x in weightVector[i]];
        weightVector[i] = weightVector[i] + learningRate*(np.matmul(predDiff, trainImages) - [l1Lambda*x for x in l1RegAdd]);

In [73]:
def updateWeight(l1):
    for i in range(labelNums):
        updateWeighti(i, l1)

In [58]:
print(weightVector[0])
updateWeight(True)
print(weightVector[0])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

In [74]:
cost = 0.0
import math
def lossFunction(inputImages, inputLabels):
    global weightVector, cost
    cost = 0.0
    for i in range(labelNums):
        predInput = np.matmul(inputImages, np.transpose(weightVector[i]))
        lnPredInput = [math.log(x) for x in predInput]
        cost += np.matmul(inputLabels, np.transpose(predInput))

In [75]:
for i in range(100):
    updateWeight(True)
    lossFunction(trainImages, trainLabels)
    print(cost)

1.92356239861e+14


ValueError: math domain error